<a href="https://colab.research.google.com/github/Yukiii0517/QM2/blob/main/QM_updated_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Method Overview
**Objective**: This study examines the relationship between gambling expenditure and labour market conditions in Australia, focusing on unemployment rates across eight regions and at the national level. The analysis proceeds in two stages:

**Panel regression** to identify robust **correlations** while controlling for unobserved regional heterogeneity.

**Granger causality analysis** to examine lead–lag dynamics and temporal **causation**.

This two-step approach allows the study to separate contemporaneous association from dynamic predictive causality.

# 1. Data
**1.1 Unit of Analysis**

Cross-sectional unit: Australian regions $i = ACT, NSW, \dots, WA$

Time unit: annual observations $t=1,\dots,T$

Balanced panel: each region observed over the same time span

**1.2 Key Variables**

*  Dependent or independent relies on the test direction

*  $Gambling_t^{(i)}$ : real per-capita gambling expenditure $(\$)$

*  $Unemp_t^{(i)}$ : regional unemployment rate $(\%)$

**1.3 Transformations**

$Gambling_t^{(i)} → ln(Gambling_t^{(i)})$

**1.4 Control variables ($X_t^{(i)}$):**

Quarterly Population Estimates (Persons)

Wage price index

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
!pip install linearmodels
from linearmodels.panel import PanelOLS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 11.3 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving QM2.csv to QM2.csv


In [ ]:
# Read data
df = pd.read_csv('QM2.csv')
df.head()

,State,Year,Real per capita total gambling expenditure value (AU$),Unemployment rate (%),Final consumption expenditure minus net loss from gambling ( Millions $),Average Week Earnings (AU$),Quarterly Population Estimates (Persons),Wage price index
0,ACT,1998–99,1739.17,6.00,11739.0,690.15,312917.25,69.25
1,ACT,1999–00,1757.75,5.04,12528.0,730.90,316057.50,70.98
2,ACT,2000–01,1761.04,4.58,12807.0,754.80,319771.00,73.50
3,ACT,2001–02,1718.74,4.52,13462.0,721.55,323407.50,75.80
4,ACT,2002–03,1723.14,4.26,14253.0,789.85,326411.25,78.40


In [ ]:
# Convert year columns
df.dropna(subset=['Year'], inplace=True)
df['year'] = df['Year'].str[-2:].astype(int) + 2000
df.loc[df['year'] > 2025, 'year'] -= 100
df

,State,Year,Real per capita total gambling expenditure value (AU$),Unemployment rate (%),Final consumption expenditure minus net loss from gambling ( Millions $),Average Week Earnings (AU$),Quarterly Population Estimates (Persons),Wage price index,year
0,ACT,1998–99,1739.17,6.00,11739.0,690.15,312917.25,69.25,1999
1,ACT,1999–00,1757.75,5.04,12528.0,730.90,316057.50,70.98,2000
2,ACT,2000–01,1761.04,4.58,12807.0,754.80,319771.00,73.50,2001
3,ACT,2001–02,1718.74,4.52,13462.0,721.55,323407.50,75.80,2002
4,ACT,2002–03,1723.14,4.26,14253.0,789.85,326411.25,78.40,2003
...,...,...,...,...,...,...,...,...,...
203,WA,2019–20,787.04,6.02,126209.0,1372.15,2695576.75,132.48,2020
204,WA,2020–21,971.28,6.09,129351.0,1400.50,2734164.50,134.43,2021
205,WA,2021–22,946.83,3.81,137505.0,1462.20,2772794.50,137.35,2022
206,WA,2022–23,925.11,3.56,144880.0,1532.65,2853711.00,142.58,2023


In [ ]:
# Relabel the columns
STATE = "State"
YEAR  = "year"
GAMB  = "Real per capita total gambling expenditure value (AU$)"
UNEMP = "Unemployment rate (%)"
WPI   = "Wage price index"
POP   = "Quarterly Population Estimates (Persons)"

# Keep only necessary columns
df = df[[STATE, YEAR, GAMB, UNEMP, WPI, POP]].copy()

# Ensure numeric data of all variables
df[[GAMB, UNEMP, WPI, POP]] = df[[GAMB, UNEMP, WPI, POP]].apply(
    pd.to_numeric, errors="coerce"
)

# Log gambling (must be > 0)
df = df[df[GAMB] > 0]
df["log_gambling"] = np.log(df[GAMB])

# Log controls (must be > 0)
df = df[df[POP] > 0]
df["log_pop"] = np.log(df[POP])

# Drop missing and set panel index
df = df.dropna(subset=[STATE, YEAR, "log_gambling", UNEMP, WPI, "log_pop"])
df = df.set_index([STATE, YEAR]).sort_index()

df


Real per capita total gambling expenditure value (AU$)  \
State year                                                           
ACT   1999                                            1739.17        
      2000                                            1757.75        
      2001                                            1761.04        
      2002                                            1718.74        
      2003                                            1723.14        
...                                                       ...        
WA    2020                                             787.04        
      2021                                             971.28        
      2022                                             946.83        
      2023                                             925.11        
      2024                                             896.41        

            Unemployment rate (%)  Wage price index  \
State year                                            
ACT   1999                   6.00             69.25   
      2000                   5.04             70.98   
      2001                   4.58             73.50   
      2002                   4.52             75.80   
      2003                   4.26             78.40   
...                           ...               ...   
WA    2020                   6.02            132.48   
      2021                   6.09            134.43   
      2022                   3.81            137.35   
      2023                   3.56            142.58   
      2024                   3.73            148.85   

            Quarterly Population Estimates (Persons)  log_gambling    log_pop  
State year                                                                     
ACT   1999                                 312917.25      7.461163  12.653694  
      2000                                 316057.50      7.471790  12.663679  
      2001                                 319771.00      7.473660  12.675360  
      2002                                 323407.50      7.449347  12.686668  
      2003                                 326411.25      7.451903  12.695913  
...                                              ...           ...        ...  
WA    2020                                2695576.75      6.668279  14.807123  
      2021                                2734164.50      6.878615  14.821336  
      2022                                2772794.50      6.853120  14.835366  
      2023                                2853711.00      6.829913  14.864131  
      2024                                2948666.75      6.798398  14.896864  

[208 rows x 6 columns]

# 3. Models
# 3.1 Time-series OLS + Panel Regression
**3.1.A Model A: Gambling leads unemployment**

*   Test of gambling → later unemployment




**Model A1: National time-series OLS (Australia as a whole)**

$$
Unemp_t
=
\alpha
+\sum_{k=0}^{p}\beta_k\,\ln(Gambling_{t-k})
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\varepsilon_t
$$

**Model A2: Single-region time-series OLS** (e.g. NSW only)

* Same equation - just applied to one region.

$$
Unemp_{s,t}
=
\alpha
+\sum_{k=0}^{p}\beta_k\,\ln(Gambling_{i,t-k})
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\varepsilon_{i,t}
$$

\begin{align}
H_0 &: \beta_0 = \beta_1 = \cdots = \beta_p = 0
&& \text{(Gambling expenditure has no association with unemployment)} \\
H_1 &: \exists \; k \in \{0,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Gambling expenditure is associated with unemployment at some lag)}
\end{align}

Both two times series OLS test for “Does gambling correlate with later unemployment within this specific region over time?”

In [ ]:
def ts_ols_modelA(ts_df, *, year_col=YEAR, p=1):

    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    # Lag gambling INCLUDING contemporaneous term (k = 0)
    for k in range(0, p + 1):
        d[f"g_L{k}"] = d["log_gambling"].shift(k)

    rhs = (
        [f"g_L{k}" for k in range(0, p + 1)]
        + [ "log_pop", WPI]
    )

    d = d.dropna(subset=[UNEMP] + rhs)

    y = d[UNEMP].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(
        cov_type="HAC",
        cov_kwds={"maxlags": p}
    )

    return res, d


In [ ]:
#Call National level
df_flat = df.reset_index()

nat = (
    df_flat
    .groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
    .mean()
    .reset_index()
)

res_nat, used_nat = ts_ols_modelA(nat, year_col=YEAR, p=5)
print(res_nat.summary())


In [ ]:
# Call National level Exclude Northern Territory
df_flat_noNT = df_flat[df_flat["State"] != "NT"].copy()
# National series excluding NT (average across remaining states)
nat_noNT = (
    df_flat_noNT
    .groupby(YEAR)[[UNEMP, "log_gambling","log_pop", WPI]]
    .mean()
    .reset_index()
)

res_nat_noNT, used_nat_noNT = ts_ols_modelA(
    nat_noNT,
    year_col=YEAR,
    p=5
)

print(res_nat_noNT.summary())


In [ ]:
#Call Regional level
region_nsw = df_flat[df_flat["State"] == "NSW"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

res_nsw, used_nsw = ts_ols_modelA(region_nsw, year_col=YEAR, p=3)
print(res_nsw.summary())

In [ ]:
# MAKE ONE RESULT TABLE (National, 8 states, National excl NT) ======

from statsmodels.iolib.summary2 import summary_col
import numpy as np
import pandas as pd

# ---------- helpers ----------
def cumulative_gambling_effect(res, p):
    return sum(float(res.params.get(f"g_L{k}", 0.0)) for k in range(0, p + 1))

def wald_pvalue_gambling_raw(res, p):
    terms = [f"g_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)  # scalar=True avoids FutureWarning
    return float(wt.pvalue)

def format_p(pval, decimals=6):
    # raw numeric p-value, formatted consistently; if extremely tiny, show scientific
    if pval == 0.0:
        return "0"
    if pval < 10**(-decimals):
        return f"{pval:.2e}"
    return f"{pval:.{decimals}f}"

# ---------- settings ----------
P = 4     # lag length used in ts_ols_modelA (must match your estimation)
EXCL = "NT"      # exclude NT for the robustness national series

# ---------- flatten panel ----------
df_flat = df.reset_index()

# ---------- National (All) ----------
nat_all = (
    df_flat.groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_all, _ = ts_ols_modelA(nat_all, year_col=YEAR, p=P)

# ---------- State models ----------
state_models, state_names = [], []
for st in sorted(df_flat["State"].dropna().unique()):
    sub = df_flat[df_flat["State"] == st][
        [YEAR, UNEMP, "log_gambling",  "log_pop", WPI]
    ]
    res, _ = ts_ols_modelA(sub, year_col=YEAR, p=P)
    state_models.append(res)
    state_names.append(st)

# ---------- National (exclude NT) ----------
nat_excl = (
    df_flat[df_flat["State"] != EXCL]
    .groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_excl, _ = ts_ols_modelA(nat_excl, year_col=YEAR, p=P)

# ---------- build summary_col table ----------
models = [model_nat_all] + state_models + [model_nat_excl]
names  = ["National (All)"] + state_names + [f"National (excl {EXCL})"]

regressor_order = (
    ["const"]
    + [f"g_L{k}" for k in range(0, P + 1)]
    + [ "log_pop", WPI]
)

# Keep info_dict minimal to avoid duplicated lines
table = summary_col(
    models,
    stars=True,
    float_format="%0.3f",
    model_names=names,
    regressor_order=regressor_order,
    info_dict={"N": lambda x: f"{int(x.nobs)}"},
)

# ---------- append extra rows (aligned as normal rows) ----------
df_table = table.tables[0]

cum_vals = [f"{cumulative_gambling_effect(m, P):.3f}" for m in models]
wald_vals = [format_p(wald_pvalue_gambling_raw(m, P), decimals=6) for m in models]

df_table.loc[f"Cumulative gambling effect (sum g_L0..g_L{P})"] = cum_vals
df_table.loc[f"Wald p-value (H0: g_L0..g_L{P} = 0)"] = wald_vals

# ---------- print in a “parallel” aligned way (avoid wrapping) ----------
with pd.option_context(
    "display.width", 4000,
    "display.max_columns", None,
    "display.max_colwidth", None
):
    print(df_table.to_string())


                                            National (All)        ACT        NSW          NT         QLD         SA        TAS        VIC         WA National (excl NT)
const                                             -969.037   335.308*   -163.523  511.535***  743.764***   762.955*    190.466     45.383    291.165        1108.630***
                                                 (590.096)  (173.306)  (119.328)   (108.024)   (224.166)  (424.944)  (153.739)  (169.144)  (702.212)          (349.699)
g_L0                                                -0.021      1.367      1.165      -0.114   -2.118***  -3.435***     -2.852  -1.451***     -2.186          -4.803***
                                                   (0.654)    (1.376)    (1.703)     (0.128)     (0.595)    (0.420)    (2.082)    (0.397)    (3.479)            (1.069)
g_L1                                             -1.627***      0.058  -8.470***    0.259***   -7.367***  -4.810***  -3.995***   1.817***     -1.960          -7

**Intepret the Coefficient**

Note: Here I deliberately set $k$ starting from $0$, so that when $k=0$ we capture the contemporaneous effect, that is, how unemployment and gambling move within the same year.

Updated: I've included the cumulative gambling effect row and Wald p-value to help the analysis.

**Model A3: 8-region panel** (main regional analysis)

$$
\ln(Unemp_{i,t})
=
\alpha
+\sum_{k=0}^{p}\theta_k\,Gambling_{i,t-k}
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\eta_{i,t},
\qquad k=0,1,\dots,p
$$

\begin{align}
H_0 &: \sum_{k=0}^{p}\theta_k = 0,
&& k = 0,1,\dots,p
&& \text{(No overall association over the lag window)} \\
H_1 &: \sum_{k=0}^{p}\theta_k \neq 0,
&& k = 0,1,\dots,p
&& \text{(Overall association over the lag window)}
\end{align}


**Note:** We only use panel regression when we have both **cross-sectional** and **time-series** data. In this case, it only works when we include all 8 regions together with their time-series data.

In the previous section, I use time-series OLS because we do not use the cross-sectional dimension and instead examine one region at a time.


Panel model tells “Within regions, does gambling lead unemployment **after accounting for regional differences and national shocks?**”

**The error terms**

**$\bf 1. \; \mu^{(i)}$ :** Region fixed effects $:=$ everything about that region that does NOT change over time

“Controls for things that make $NSW$ permanently different from $WA$.”

Examples: Cultural attitudes toward gambling, Long-run industry structure (e.g. mining-heavy $WA$), Historical labour-market characteristics, etc.

These factors differ across regions but are constant over time within each region.

**$\bf 2. \; \lambda_t$ :** Year fixed effects $:=$ everything that affects all regions in year

“Controls for things that affect all regions in a given year, like COVID.”

Examples: National recessions / booms, COVID, Interest-rate cycles, etc.

So if unemployment and gambling both spike in 2020: that variation is absorbed by 𝜆 not attributed to gambling causing unemployment

**$\bf 3. \; \epsilon_t^{(i)}$ :** Idiosyncratic shocks $:=$ unexpected region-year noise

“Everything else we can't explain.”

Examples: A local factory closure, A temporary regional event, Measurement error, etc.

In [ ]:
def panel_fe_modelA(df_panel, *, p=1):

    d = df_panel.copy()

    # Ensure MultiIndex (STATE, YEAR)
    if not isinstance(d.index, pd.MultiIndex):
        d = d.set_index([STATE, YEAR]).sort_index()
    else:
        d = d.sort_index()

    # Required columns (must exist in d)
    required = [UNEMP, "log_gambling", "log_pop", WPI]
    missing = [c for c in required if c not in d.columns]
    if missing:
        raise KeyError(f"Missing columns in df_panel: {missing}")

    # Build gambling lags within state: k = 0..p
    for k in range(0, p + 1):
        d[f"g_L{k}"] = d.groupby(level=0)["log_gambling"].shift(k)

    # Regressors: gambling lags + controls
    rhs = [f"g_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]

    # Keep only necessary cols and drop missing
    d = d[[UNEMP] + rhs].dropna()

    y = d[UNEMP].astype(float)
    X = d[rhs].astype(float)

    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    res = mod.fit(cov_type="clustered", cluster_entity=True)
    return res


In [ ]:
res_panel = panel_fe_modelA(df, p=5)

print(res_panel.summary)

                            PanelOLS Estimation Summary                            
Dep. Variable:     Unemployment rate (%)   R-squared:                        0.2443
Estimator:                      PanelOLS   R-squared (Between):             -28.046
No. Observations:                    168   R-squared (Within):              -5.3992
Date:                   Tue, Dec 30 2025   R-squared (Overall):             -27.427
Time:                           09:39:15   Log-likelihood                   -108.05
Cov. Estimator:                Clustered                                           
                                           F-statistic:                      5.3348
Entities:                              8   P-value                           0.0000
Avg Obs:                          21.000   Distribution:                   F(8,132)
Min Obs:                          21.000                                           
Max Obs:                          21.000   F-statistic (robust):         -2.

**3.1.B Model B: Gambling lags unemployment**

Test of unemployment → later gambling




This part should be really easy to understand if you understand 3.1.A, we are just switching the dependent and independent variables.
Therefore, I'll simplify the explanation ;)



**Model B1 and B2: Time-series OLS run at National and Regional level**

$$
\ln(Gambling_{i,t})
=
\alpha
+\sum_{k=0}^{p}\theta_k\,Unemp_{i,t-k}
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\mu_i
+\lambda_t
+\eta_{i,t}
$$

\begin{align}
H_0 &: \beta_0 = \beta_1 = \cdots = \beta_p = 0,
\qquad k \in \{0,1,\dots,p\}
&& \text{(Gambling expenditure has no association with unemployment)} \\
H_1 &: \exists\, k \in \{0,1,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Gambling expenditure is associated with unemployment over the lag window)}
\end{align}


In [ ]:
# Model B (time-series OLS): log_gambling_t on lags of unemployment + controls (log_awe, log_pop, WPI)

def ts_ols_modelB(ts_df, *, year_col=YEAR, p=1):
    """
    log_gambling_t ~ u_{t}, u_{t-1}, ..., u_{t-p} + log_awe_t + log_pop_t + WPI_t
    HAC (Newey–West) SEs with maxlags = p
    """

    d = (
        ts_df[[year_col, UNEMP, "log_gambling", "log_pop", WPI]]
        .dropna()
        .sort_values(year_col)
        .copy()
    )

    # Lag unemployment: k = 0..p
    for k in range(0, p + 1):
        d[f"u_L{k}"] = d[UNEMP].shift(k)

    rhs = [f"u_L{k}" for k in range(0, p + 1)] + ["log_pop", WPI]
    d = d.dropna(subset=["log_gambling"] + rhs)

    y = d["log_gambling"].astype(float)
    X = sm.add_constant(d[rhs].astype(float))

    res = sm.OLS(y, X).fit(cov_type="HAC", cov_kwds={"maxlags": p})
    return res, d


In [ ]:
# Call Model B1
# National
df_flat = df.reset_index()
nat = df_flat.groupby(YEAR)[[UNEMP, "log_gambling", "log_pop", WPI]].mean().reset_index()

res_nat_B, used_nat_B = ts_ols_modelB(nat, year_col=YEAR, p=5)# change p here
print(res_nat_B.summary())

In [ ]:
#Call Model B2
# Regional
region = df_flat[df_flat["State"] == "NSW"][[YEAR, UNEMP, "log_gambling", "log_pop", WPI]] # change region here

res_nsw_B, used_nsw_B = ts_ols_modelB(region, year_col=YEAR, p=3)# change p here
print(res_nsw_B.summary())

In [ ]:
# ========= MODEL B RESULTS TABLE (National + 8 states + National excl NT)


# ---- helpers ----
def cumulative_unemp_effect(res, p):
    return sum(float(res.params.get(f"u_L{k}", 0.0)) for k in range(0, p + 1))

def wald_pvalue_unemp_block(res, p):
    terms = [f"u_L{k}" for k in range(0, p + 1)]
    hypothesis = " = ".join(terms) + " = 0"
    wt = res.wald_test(hypothesis, scalar=True)  # avoids FutureWarning
    return float(wt.pvalue)

def fmt_p(pval, decimals=6):
    # raw p-value, readable (scientific if very small)
    if pval == 0.0:
        return "0"
    if pval < 10**(-decimals):
        return f"{pval:.2e}"
    return f"{pval:.{decimals}f}"

# ---- settings ----
P = 5       # set lag length here (e.g., 3 if you want u_L0..u_L3)
EXCL = "NT"    # exclude NT for robustness national series

# ---- flatten ----
df_flat = df.reset_index()

# ---- National (All) ----
nat_all = (
    df_flat.groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_all, _ = ts_ols_modelB(nat_all, year_col=YEAR, p=P)

# ---- Each state ----
state_models, state_names = [], []
for st in sorted(df_flat["State"].dropna().unique()):
    sub = df_flat[df_flat["State"] == st][
        [YEAR, UNEMP, "log_gambling",  "log_pop", WPI]
    ]
    res, _ = ts_ols_modelB(sub, year_col=YEAR, p=P)
    state_models.append(res)
    state_names.append(st)

# ---- National (exclude NT) ----
nat_excl = (
    df_flat[df_flat["State"] != EXCL]
    .groupby(YEAR)[[UNEMP, "log_gambling",  "log_pop", WPI]]
    .mean()
    .reset_index()
)
model_nat_excl, _ = ts_ols_modelB(nat_excl, year_col=YEAR, p=P)

# ---- Build table ----
models = [model_nat_all] + state_models + [model_nat_excl]
names  = ["National (All)"] + state_names + [f"National (excl {EXCL})"]

regressor_order = (
    ["const"]
    + [f"u_L{k}" for k in range(0, P + 1)]
    + [ "log_pop", WPI]
)

table = summary_col(
    models,
    stars=True,
    float_format="%0.3f",
    model_names=names,
    regressor_order=regressor_order,
    info_dict={
        "R-squared": lambda x: f"{x.rsquared:.3f}",
        "R-squared Adj.": lambda x: f"{x.rsquared_adj:.3f}",
        "N": lambda x: f"{int(x.nobs)}",
    },
)

# ---- Append extra rows ----
df_table = table.tables[0]

cum_vals  = [f"{cumulative_unemp_effect(m, P):.3f}" for m in models]
wald_pvals = [fmt_p(wald_pvalue_unemp_block(m, P), decimals=6) for m in models]

df_table.loc[f"Cumulative unemployment effect (sum u_L0..u_L{P})"] = cum_vals
df_table.loc[f"Wald p-value (H0: u_L0..u_L{P} = 0)"] = wald_pvals

# ---- Print aligned ----
with pd.option_context("display.width", 4000, "display.max_columns", None, "display.max_colwidth", None):
    print(df_table.to_string())


                                                National (All)         ACT       NSW         NT        QLD         SA        TAS       VIC        WA National (excl NT)
const                                                   60.675  -64.015***   58.888*  -147.379*   63.458**      5.738     -7.756    31.174   -43.529          76.307***
                                                      (42.150)    (20.356)  (32.142)   (88.193)   (25.874)   (58.603)   (13.881)  (25.985)  (32.082)           (28.562)
u_L0                                                   -0.053*    -0.085**   -0.040*      0.050  -0.040***  -0.081***  -0.056***  -0.139**    -0.018          -0.070***
                                                       (0.030)     (0.042)   (0.021)    (0.117)    (0.015)    (0.024)    (0.018)   (0.058)   (0.017)            (0.013)
u_L1                                                     0.031       0.050   0.035**      0.012     -0.004     -0.004     -0.038    0.044*    -0.018            

**Model B3: 8-region panel**

$$
\ln(Gambling_{i,t})
=
\alpha
+\sum_{k=0}^{p}\theta_k\,Unemp_{i,t-k}
+\delta_2 \ln(Pop_{i,t})
+\delta_3 WPI_{i,t}
+\mu_i
+\lambda_t
+\eta_{i,t}
$$

\begin{align}
H_0 &: \sum_{k=0}^{p}\theta_k = 0,
&& k = 0,1,\dots,p
&& \text{(No overall association over the lag window)} \\
H_1 &: \sum_{k=0}^{p}\theta_k \neq 0,
&& k = 0,1,\dots,p
&& \text{(Overall association over the lag window)}
\end{align}

In [ ]:
# Model B (panel FE): log_gambling on lags of unemployment + controls (log_awe, log_pop, WPI)

def panel_modelB(df_panel, *, p=1):


    d = df_panel.copy()

    # Ensure MultiIndex (STATE, YEAR)
    if not isinstance(d.index, pd.MultiIndex):
        d = d.set_index([STATE, YEAR]).sort_index()
    else:
        d = d.sort_index()

    # Lags of unemployment within state: k = 0..p
    for k in range(0, p + 1):
        d[f"u_L{k}"] = d.groupby(level=0)[UNEMP].shift(k)

    # Regressors: unemployment lags + controls
    rhs = [f"u_L{k}" for k in range(0, p + 1)] + [ "log_pop", WPI]

    # Keep complete cases
    d = d.dropna(subset=["log_gambling"] + rhs)

    y = d["log_gambling"].astype(float)
    X = d[rhs].astype(float)

    mod = PanelOLS(y, X, entity_effects=True, time_effects=True)
    return mod.fit(cov_type="clustered", cluster_entity=True)


In [ ]:
# call panel
resB_panel = panel_modelB(df, p=5)
print(resB_panel.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:           log_gambling   R-squared:                        0.2896
Estimator:                   PanelOLS   R-squared (Between):             -2.7141
No. Observations:                 168   R-squared (Within):              -1.1889
Date:                Tue, Dec 30 2025   R-squared (Overall):             -2.7120
Time:                        08:49:15   Log-likelihood                    25.638
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6.7274
Entities:                           8   P-value                           0.0000
Avg Obs:                       21.000   Distribution:                   F(8,132)
Min Obs:                       21.000                                           
Max Obs:                       21.000   F-statistic (robust):           1.16e+14
                            

All of the models in Section 3.1 only provide information about **correlation**. To further examine whether a **causal** relationship exists, we employ Granger causality analysis.

What “Granger causality” actually means is not philosophical causality. Instead, it asks a specific and testable question:

Do past values of gambling improve our ability to predict unemployment, over and above unemployment’s own past values?

If yes, gambling is said to Granger-cause unemployment.

But the good news is that Granger causality is much simpler than Section 3.1, where we used combined OLS and panel regression. For the panel analysis, we use the same equation and model structure for both national-level and regional-level data, and we simply switch the dependent and independent variables.

# 3.2 Conditional Granger causality tests

3.2.C Model C: Gambling Granger-causes unemployment
* supports gambling leads unemployment

$$
Unemp_t
=
\alpha
+\sum_{k=1}^{p}\gamma_k\,Unemp_{t-k}
+\sum_{k=1}^{p}\beta_k\,\ln(Gambling_{t-k})
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\epsilon_t
$$

**Hypothesis**:


\begin{align}
H_0 &: \beta_1 = \beta_2 = \cdots = \beta_p = 0
&& \text{(Lagged gambling has no predictive content for unemployment)} \\
H_1 &: \exists\, k \in \{1,\dots,p\} \text{ such that } \beta_k \neq 0
&& \text{(Lagged gambling has predictive content for unemployment)}
\end{align}

In [ ]:
def granger_gambling_to_unemp_joint(
    ts_df,
    *,
    year_col=YEAR,
    maxlag=4,
    controls=("log_pop", WPI)
):
    """
    Controlled Granger-style analysis:
      UNEMP_t ~ UNEMP_{t-1..t-L} + log_gambling_{t-1..t-L} + controls_t

    Output per lag:
      - cumulative gambling coefficient (direction)
      - joint F-statistic
      - joint p-value
      - significance stars (JOINT ONLY)
    """

    def stars(p):
        if p < 0.01:
            return "***"
        if p < 0.05:
            return "**"
        if p < 0.1:
            return "*"
        return ""

    cols = [year_col, UNEMP, "log_gambling", *controls]
    d = ts_df[cols].dropna().sort_values(year_col).copy()

    rows = []

    for L in range(1, maxlag + 1):
        tmp = d.copy()

        # build lags
        for k in range(1, L + 1):
            tmp[f"u_L{k}"] = tmp[UNEMP].shift(k)
            tmp[f"g_L{k}"] = tmp["log_gambling"].shift(k)

        tmp = tmp.dropna()

        y = tmp[UNEMP].astype(float)

        X = sm.add_constant(
            tmp[[f"u_L{k}" for k in range(1, L + 1)] +
                [f"g_L{k}" for k in range(1, L + 1)] +
                list(controls)]
            .astype(float)
        )

        res = sm.OLS(y, X).fit()

        # joint F-test: all gambling lags = 0
        param_names = list(X.columns)
        R = np.zeros((L, len(param_names)))
        for i, k in enumerate(range(1, L + 1)):
            R[i, param_names.index(f"g_L{k}")] = 1.0

        ftest = res.f_test(R)

        # cumulative direction
        cum_coef = sum(res.params[f"g_L{k}"] for k in range(1, L + 1))

        rows.append({
            "Lag": L,
            "Cumulative gambling effect": cum_coef,
            "Direction": "Positive" if cum_coef > 0 else "Negative",
            "F_stat": float(ftest.fvalue),
            "p_value": float(ftest.pvalue),
            "N": int(res.nobs)
        })

    return pd.DataFrame(rows)


In [ ]:
#national
gc_nat = granger_gambling_to_unemp_joint(
    nat_all,
    year_col=YEAR,
    maxlag=5
)
print(gc_nat)


   Lag  Cumulative gambling effect Direction    F_stat   p_value   N
0    1                    0.437768  Positive  0.094208  0.762067  25
1    2                    2.831183  Positive  4.920045  0.020614  24
2    3                    2.075466  Positive  5.397357  0.011155  23
3    4                    2.391649  Positive  5.497764  0.011103  22
4    5                   -0.508097  Negative  3.883495  0.043962  21


In [ ]:
#regional
# Corrected: Use .isin() for selecting multiple states
region = df_flat[df_flat["State"] == "WA"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

gc_nsw = granger_gambling_to_unemp_joint(
    region,
    year_col=YEAR,
    maxlag=5
)
print(gc_nsw)

   Lag  Cumulative gambling effect Direction    F_stat   p_value   N
0    1                    1.309308  Positive  0.482438  0.495310  25
1    2                    2.263527  Positive  2.773132  0.090716  24
2    3                    2.907799  Positive  1.414764  0.280092  23
3    4                    8.800771  Positive  1.329718  0.319064  22
4    5                   24.948622  Positive  2.054947  0.174350  21


3.2.D Model D2: Unemployment Granger-causes gambling
* supports gambling lags unemployment

$$
\ln(Gambling_t)
=
\alpha
+\sum_{k=1}^{p}\phi_k\,\ln(Gambling_{t-k})
+\sum_{k=1}^{p}\theta_k\,Unemp_{t-k}
+\delta_2 \ln(Pop_t)
+\delta_3 WPI_t
+\eta_t
$$

\begin{align}
H_0 &: \theta_1 = \theta_2 = \cdots = \theta_p = 0
&& \text{(Lagged unemployment has no predictive content for gambling)} \\
H_1 &: \exists\, k \in \{1,\dots,p\} \text{ such that } \theta_k \neq 0
&& \text{(Lagged unemployment has predictive content for gambling)}
\end{align}

In [ ]:
def granger_unemp_to_gambling_joint(
    ts_df,
    *,
    year_col=YEAR,
    maxlag=4,
    controls=( "log_pop", WPI)
):

    cols = [year_col, UNEMP, "log_gambling", *controls]
    d = ts_df[cols].dropna().sort_values(year_col).copy()

    rows = []

    for L in range(1, maxlag + 1):
        tmp = d.copy()

        # build lags
        for k in range(1, L + 1):
            tmp[f"g_L{k}"] = tmp["log_gambling"].shift(k)
            tmp[f"u_L{k}"] = tmp[UNEMP].shift(k)

        tmp = tmp.dropna()

        y = tmp["log_gambling"].astype(float)

        X = sm.add_constant(
            tmp[[f"g_L{k}" for k in range(1, L + 1)] +
                [f"u_L{k}" for k in range(1, L + 1)] +
                list(controls)]
            .astype(float)
        )

        res = sm.OLS(y, X).fit()

        # joint F-test: all unemployment lags = 0
        param_names = list(X.columns)
        R = np.zeros((L, len(param_names)))
        for i, k in enumerate(range(1, L + 1)):
            R[i, param_names.index(f"u_L{k}")] = 1.0

        ftest = res.f_test(R)

        # cumulative direction
        cum_coef = sum(res.params[f"u_L{k}"] for k in range(1, L + 1))

        rows.append({
            "Lag": L,
            "Cumulative unemployment effect": cum_coef,
            "Direction": "Positive" if cum_coef > 0 else "Negative",
            "F_stat": float(ftest.fvalue),
            "p_value": float(ftest.pvalue),
            "N": int(res.nobs)
        })

    return pd.DataFrame(rows)


In [ ]:
gd_nat = granger_unemp_to_gambling_joint(
    nat_all,
    year_col=YEAR,
    maxlag=5
)
print(gd_nat)


   Lag  Cumulative unemployment effect Direction    F_stat   p_value   N
0    1                        0.010825  Positive  0.196174  0.662581  25
1    2                        0.015762  Positive  0.159809  0.853572  24
2    3                        0.006906  Positive  0.361098  0.782052  23
3    4                        0.006983  Positive  0.135050  0.966005  22
4    5                       -0.170249  Negative  0.434956  0.812861  21


In [ ]:
region = df_flat[df_flat["State"] == "TAS"][
    [YEAR, UNEMP, "log_gambling", "log_pop", WPI]
]

gd_nsw = granger_unemp_to_gambling_joint(
    region,
    year_col=YEAR,
    maxlag=5
)
print(gd_nsw)


   Lag  Cumulative unemployment effect Direction    F_stat   p_value   N
0    1                       -0.054994  Negative  4.960121  0.037587  25
1    2                       -0.044113  Negative  0.821952  0.456302  24
2    3                       -0.067403  Negative  1.114190  0.376527  23
3    4                       -0.195851  Negative  1.362224  0.308642  22
4    5                       -0.214709  Negative  0.727359  0.622223  21


To summarise, we have two parts in our method.

Part $1$ tests for correlation using time-series OLS (national and regional) and panel regression (national).

Part $2$ tests for causation using Granger causality (national and regional).

Please tell me anything you think may be we should add or delete:), also here are some things to think about:


I am thinking about may be choose one from Times-series OLS at the national level and panel regression at the national level, maybe we should subtract the OLS at national level?
